# Capstone Project - The Battle of Neighborhoods (Week 5- Code).

In [2]:
# Import the necessary libraries.
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         375 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates

In [3]:
#Download the data.
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
# Isolate the relevant data from the features key and look at the first row.
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [6]:
# Turn data into Pandas dataframe.
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
neighborhoods.shape

(306, 4)

In [12]:
# Confirm the number of boroghs and neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    ))

The dataframe has 5 boroughs and 306 neighborhoods.


In [23]:
# Use geopy to get lat/long values for NYC.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [24]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [90]:
# Isolate the Manhattan and Brooklyn data.
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [25]:
manhattan_data.shape

(40, 4)

In [17]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [45]:
brooklyn_data.shape

(70, 4)

In [22]:
frames = [manhattan_data, brooklyn_data]
manbrook_data = pd.concat(frames)
manbrook_data.shape

(110, 4)

In [26]:
# Define Foursquare credentials.
CLIENT_ID = 'QU2DZ3KZJ5S12DQJO2M1DLCMRZDYAH220ITFMGFAODIMRAA0'
CLIENT_SECRET = 'IUCTSMMMVB1D2TJZK3X21QKGNW2MVVDEP5RLH0YJKZGQRKV5'
VERSION = '20180605' 
LIMIT = 100 

In [50]:
# Lets explore the Upper West side where Mr Riccis restaurant is located. This area is located on row index 12.
neighborhood_latitude = manhattan_data.loc[12, 'Latitude'] 
neighborhood_longitude = manhattan_data.loc[12, 'Longitude'] 

neighborhood_name = manhattan_data.loc[12, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Upper West Side are 40.787657998534854, -73.97705923630603.


In [51]:
# Identify the top 100 venues within a 100m radius.
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [38]:
# View the results. (Long JSON file)
#results

In [52]:
# Function to extract venue.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
# Turn JSON file to pd dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Jacob's Pickles,Southern / Soul Food Restaurant,40.786653,-73.975622
1,Eléa,Greek Restaurant,40.787531,-73.976681
2,AMC Loews 84th Street 6,Movie Theater,40.786770,-73.977608
3,Maison Pickle,American Restaurant,40.786990,-73.977787
4,Tiki Chick,Tiki Bar,40.786854,-73.975410


In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [55]:
# Repeat for the whole manbrook_data df.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
manbrook_venues = getNearbyVenues(names=manbrook_data['Neighborhood'],
                                   latitudes=manbrook_data['Latitude'],
                                   longitudes=manbrook_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine

In [46]:
print(manbrook_venues.shape)
manbrook_venues.head()

(5506, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
4,Marble Hill,40.876551,-73.91066,Subway,40.874667,-73.909586,Sandwich Place


In [47]:
manbrook_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,49,49,49,49,49,49
Battery Park City,62,62,62,62,62,62
Bay Ridge,73,73,73,73,73,73
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,31,31,31,31,31,31
Bergen Beach,7,7,7,7,7,7
Boerum Hill,85,85,85,85,85,85
Borough Park,20,20,20,20,20,20
Brighton Beach,40,40,40,40,40,40


In [57]:
print('There are {} uniques categories.'.format(len(manbrook_venues['Venue Category'].unique())))

There are 382 uniques categories.


In [58]:
# One-hot encoding.
manbrook_onehot = pd.get_dummies(manbrook_venues[['Venue Category']], prefix="", prefix_sep="")

manbrook_onehot['Neighborhood'] = manbrook_venues['Neighborhood'] 

fixed_columns = [manbrook_onehot.columns[-1]] + list(manbrook_onehot.columns[:-1])
manbrook_onehot = manbrook_onehot[fixed_columns]

manbrook_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Gym,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pila

In [62]:
# Fixing the Neighborhood colun to be in the correct position.
#col_name="Neighborhood"
#first_col = manbrook_onehot.pop(col_name)

In [64]:
manbrook_onehot.insert(0, col_name, first_col)

In [65]:
manbrook_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Gym,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pila

In [66]:
manbrook_onehot.shape

(5506, 382)

In [67]:
manbrook_grouped = manbrook_onehot.groupby('Neighborhood').mean().reset_index()
manbrook_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Gym,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pila

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
# Create df with top 10 most common venues for each area.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manbrook_grouped['Neighborhood']

for ind in np.arange(manbrook_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manbrook_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Cantonese Restaurant,Fast Food Restaurant,Pizza Place,Gas Station,Sushi Restaurant,Donut Shop,Bubble Tea Shop,Surf Spot
1,Battery Park City,Park,Coffee Shop,Hotel,Pizza Place,Gym,Memorial Site,Food Court,Gourmet Shop,Playground,Plaza
2,Bay Ridge,Italian Restaurant,Chinese Restaurant,Pizza Place,Grocery Store,American Restaurant,Greek Restaurant,Spa,Bar,Bagel Shop,Playground
3,Bedford Stuyvesant,Bar,Pizza Place,Coffee Shop,Café,Bus Stop,Italian Restaurant,Boutique,Gift Shop,Thrift / Vintage Store,Bagel Shop
4,Bensonhurst,Chinese Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant,Donut Shop,Ice Cream Shop,Flower Shop,Spa,Noodle House,Pet Store


In [70]:
neighborhoods_venues_sorted.shape

(110, 11)

In [77]:
# Now to run K-means clustering.
kclusters = 6

manbrook_grouped_clustering = manbrook_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manbrook_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 0, 0, 5, 3, 0, 5, 5, 5], dtype=int32)

In [80]:
#Unblock the following code. Only blocked because it can only be run one before producing an error.
neighborhoods_venues_sorted.pop('Cluster Labels') # added so that i could rerun this code with mutliple K values.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manbrook_merged = manbrook_data

manbrook_merged = manbrook_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manbrook_merged.tail()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Brooklyn,Dumbo,40.703176,-73.988753,3,Park,Scenic Lookout,Italian Restaurant,Bakery,Bookstore,Boxing Gym,Gym,Ice Cream Shop,Yoga Studio,Pharmacy
66,Brooklyn,Homecrest,40.598525,-73.959185,5,Bank,Chinese Restaurant,Donut Shop,Pizza Place,Cosmetics Shop,Mexican Restaurant,Sushi Restaurant,Sandwich Place,Grocery Store,Russian Restaurant
67,Brooklyn,Highland Park,40.681999,-73.890346,5,Deli / Bodega,Latin American Restaurant,Grocery Store,Pizza Place,Fried Chicken Joint,Food,Furniture / Home Store,Garden,Metro Station,Tennis Court
68,Brooklyn,Madison,40.609378,-73.948415,3,Bagel Shop,Italian Restaurant,Dessert Shop,Restaurant,Pizza Place,Pilates Studio,Candy Store,Deli / Bodega,Hotel,Food Stand
69,Brooklyn,Erasmus,40.646926,-73.948177,2,Caribbean Restaurant,Pharmacy,Bank,Convenience Store,Mobile Phone Shop,Donut Shop,Gas Station,Chinese Restaurant,Fast Food Restaurant,Lounge


In [81]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manbrook_merged['Latitude'], manbrook_merged['Longitude'], manbrook_merged['Neighborhood'], manbrook_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Lets look at each cluster more carefully.

In [82]:
# Cluster 1.
# THIS CLUSTER CONTAINS THE UPPER WEST SIDE LOCATION WHER MR RICCIS FIRST RESTAURANT IS LOCATED
# THEREFORE THIS IS THE CLUSTER OF MOST IMPORTANCE.
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 0, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Ice Cream Shop,Dessert Shop,Spa,American Restaurant,Salon / Barbershop,Noodle House,Greek Restaurant
5,Manhattanville,Italian Restaurant,Mexican Restaurant,Bank,Sushi Restaurant,Coffee Shop,Bar,Gastropub,Chinese Restaurant,Indian Restaurant,Japanese Curry Restaurant
6,Central Harlem,African Restaurant,Sandwich Place,Bar,Gym / Fitness Center,French Restaurant,American Restaurant,Seafood Restaurant,Tapas Restaurant,Bank,Gym
8,Upper East Side,Italian Restaurant,Spa,Bakery,Exhibit,American Restaurant,Coffee Shop,Hotel,Juice Bar,Yoga Studio,Wine Shop
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Diner,Bar,Deli / Bodega,Supplement Shop,Thai Restaurant
10,Lenox Hill,Italian Restaurant,Pizza Place,Bank,Sushi Restaurant,Café,Gym / Fitness Center,Gym,Burger Joint,Deli / Bodega,Cocktail Bar
12,Upper West Side,Italian Restaurant,Bakery,Café,Wine Bar,Coffee Shop,Ice Cream Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Pizza Place,Vegetarian / Vegan Restaurant
13,Lincoln Square,Plaza,Café,Concert Hall,Theater,Performing Arts Venue,Gym / Fitness Center,Bakery,Italian Restaurant,Coffee Shop,French Restaurant
14,Clinton,Coffee Shop,Theater,Sandwich Place,American Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,Indie Theater,Spa,French Restaurant
15,Midtown,Clothing Store,Hotel,Spa,Theater,Bakery,Sushi Restaurant,Food Truck,Bookstore,Coffee Shop,Sandwich Place


In [83]:
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 1, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Paerdegat Basin,Child Care Service,Bus Line,Food,Asian Restaurant,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [84]:
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 2, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Deli / Bodega,Caribbean Restaurant,Juice Bar,Coffee Shop,Donut Shop,Supermarket,Mobile Phone Shop,Middle Eastern Restaurant,Mexican Restaurant,Chinese Restaurant
10,East Flatbush,Chinese Restaurant,Moving Target,Liquor Store,Department Store,Pharmacy,Park,Caribbean Restaurant,Fast Food Restaurant,Supermarket,Print Shop
26,East New York,Deli / Bodega,Pharmacy,Pizza Place,Food Truck,Fast Food Restaurant,Metro Station,Caribbean Restaurant,Gym,Event Service,Chinese Restaurant
28,Canarsie,Deli / Bodega,Caribbean Restaurant,Gym,Bus Line,Grocery Store,Asian Restaurant,Filipino Restaurant,Factory,Falafel Restaurant,Farm
29,Flatlands,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Caribbean Restaurant,Video Store,Bar,Paper / Office Supplies Store,Chinese Restaurant,Lounge,Deli / Bodega
47,Prospect Park South,Caribbean Restaurant,Clothing Store,Mobile Phone Shop,Fast Food Restaurant,Grocery Store,Department Store,Kids Store,Fried Chicken Joint,Donut Shop,Pharmacy
56,Rugby,Bank,Caribbean Restaurant,Sandwich Place,Mobile Phone Shop,Pharmacy,Chinese Restaurant,Seafood Restaurant,Salon / Barbershop,Gas Station,Grocery Store
57,Remsen Village,Caribbean Restaurant,Fried Chicken Joint,Fish Market,Sandwich Place,Library,Fast Food Restaurant,Donut Shop,Café,Gas Station,Gym
58,New Lots,Grocery Store,Pharmacy,Pizza Place,Chinese Restaurant,Intersection,Breakfast Spot,Bank,Fried Chicken Joint,Asian Restaurant,Park
69,Erasmus,Caribbean Restaurant,Pharmacy,Bank,Convenience Store,Mobile Phone Shop,Donut Shop,Gas Station,Chinese Restaurant,Fast Food Restaurant,Lounge


In [85]:
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 3, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Coffee Shop,Deli / Bodega,Outdoors & Recreation,Kosher Restaurant,Gym / Fitness Center,Gym,Sandwich Place,Scenic Lookout,Metro Station
28,Battery Park City,Park,Coffee Shop,Hotel,Pizza Place,Gym,Memorial Site,Food Court,Gourmet Shop,Playground,Plaza
37,Stuyvesant Town,Park,Yoga Studio,Coffee Shop,Gas Station,Boat or Ferry,Bistro,Farmers Market,Bar,Harbor / Marina,Pet Service
6,Sheepshead Bay,Turkish Restaurant,Ice Cream Shop,Dessert Shop,Italian Restaurant,Harbor / Marina,Restaurant,Pizza Place,Park,Outlet Store,Miscellaneous Shop
35,Dyker Heights,Bagel Shop,Dance Studio,Burger Joint,Grocery Store,Plaza,Bus Station,Park,Filipino Restaurant,Exhibit,Factory
45,Bergen Beach,Harbor / Marina,Playground,Baseball Field,Athletics & Sports,Hockey Field,Park,Fast Food Restaurant,Fountain,Event Space,Exhibit
61,Fulton Ferry,Park,Ice Cream Shop,Pizza Place,Coffee Shop,Bakery,Clothing Store,Scenic Lookout,Roof Deck,Burger Joint,Roller Rink
65,Dumbo,Park,Scenic Lookout,Italian Restaurant,Bakery,Bookstore,Boxing Gym,Gym,Ice Cream Shop,Yoga Studio,Pharmacy
68,Madison,Bagel Shop,Italian Restaurant,Dessert Shop,Restaurant,Pizza Place,Pilates Studio,Candy Store,Deli / Bodega,Hotel,Food Stand


In [86]:
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 4, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Locksmith,Pool,Yemeni Restaurant,Fish & Chips Shop,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [87]:
manbrook_merged.loc[manbrook_merged['Cluster Labels'] == 5, manbrook_merged.columns[[1] + list(range(5, manbrook_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Yoga Studio,Bakery,Tennis Stadium,Supplement Shop,Storage Facility,Steakhouse,Seafood Restaurant,Clothing Store,Coffee Shop
2,Washington Heights,Café,Mobile Phone Shop,Pizza Place,Grocery Store,Chinese Restaurant,Bakery,Bank,Spanish Restaurant,Donut Shop,Coffee Shop
3,Inwood,Mexican Restaurant,Café,Caribbean Restaurant,Bakery,Wine Bar,Bank,Pizza Place,Lounge,Park,Deli / Bodega
4,Hamilton Heights,Pizza Place,Mexican Restaurant,Sandwich Place,Café,Yoga Studio,Cocktail Bar,Indian Restaurant,Juice Bar,Coffee Shop,Caribbean Restaurant
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Deli / Bodega,Sandwich Place,Steakhouse,Bank,Chinese Restaurant,Seafood Restaurant,Café
20,Lower East Side,Pizza Place,Chinese Restaurant,Café,Art Gallery,Sandwich Place,Bakery,Pharmacy,Theater,Grocery Store,Tailor Shop
1,Bensonhurst,Chinese Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant,Donut Shop,Ice Cream Shop,Flower Shop,Spa,Noodle House,Pet Store
2,Sunset Park,Bank,Mobile Phone Shop,Gym,Mexican Restaurant,Bakery,Fried Chicken Joint,Deli / Bodega,Latin American Restaurant,Pizza Place,Pharmacy
4,Gravesend,Italian Restaurant,Lounge,Pizza Place,Martial Arts School,Arts & Crafts Store,Furniture / Home Store,Bus Station,Baseball Field,Bar,Sporting Goods Shop
5,Brighton Beach,Russian Restaurant,Sushi Restaurant,Restaurant,Bank,Donut Shop,Gourmet Shop,Beach,Eastern European Restaurant,Mobile Phone Shop,Pharmacy
